# 9장 파서 조합기 라이브러리

* 싸이그래머 / 스칼라ML - 함수형 스칼라 파트
* 김무성

# 차례

* 9.1 대수의 설계 : 첫 시도
* 9.2 가능한 대수 하나
* 9.3 문맥 민감성 처리
* 9.4 JSON 파서 작성
* 9.5 오류 보고
* 9.6 대수의 구현
* 9.7 요약

# 9.1 대수의 설계 : 첫 시도

#### 하고 싶은 것

* "abracadabra"에 'abra'가 있는가? -> yes

-----

#### 하고 싶은 것
* 'a'나 "abra"가 있는지 찾아내는 파서를 생성하고 싶다

-----

In [ ]:
// char('a') -> Parser('a') 생성
def char(c: Char): Parser[Char]

In [ ]:
// run(char('a'))('a') -> 'a'
def run[A](p: Parser[A])(input: String): Either[ParseError, A]

In [ ]:
trait Parsers[ParseError, Parser[+_]] {
    def run[A](p: Parser[A])(input: String): Either[ParseError, A]
    def char(c: Char): Parser[Char]
}

In [ ]:
// run(char('a'))('a') == Right('a')
run(char(c))(c.toString) == Right(c)

-----

In [ ]:
// string("abra") -> Parser("abra") 생성
def string(s: String): Parser[String]

In [ ]:
// run(string("abra"))("abra") == Right("abra")
run(string(s))(s) == Right(s)

-----

#### 하고 싶은 것
* "abracadabra"에 
    * "abra" 또는(or) "cadabra"가 있는가? 
    * -> yes

-----

In [ ]:
// orString("abr", "cadabra") -> Pareser 생성
def orString(s1: String, s2: String): Parser[String]

-------

In [ ]:
// or(string("abra"), string("cadabra")) -> Parser 생성
def or[A](s1: Parser[A], s2: Parser[A]): Parser[A]

In [ ]:
run(or(string("abra"),string("cadabra")))("abra") == Right("abra")

In [ ]:
run(or(string("abra"),string("cadabra")))("cadabra") == Right("cadabra")

----------

#### 하고 싶은 것
* or(string("abra"), string("cadabra")) 를 아래처럼도 쓰고 싶다
* "abra" | "cadabra"

-----

#### 목록 9.1 파서에 중위 구문 추가

In [ ]:
trait Parsers[ParseError, Parser[+_]] { self =>
    
    def or[A](s1: Parser[A], s2: Parser[A]): Parser[A]
    implicit def string(s: String): Parser[String]
    implicit def operators[A](p: Parser[A]) = ParserOps[A](p)
    implicit def asStringParser[A](a: A)(implicit f: A => Parser[String]) : 
        ParserOps[String] = ParserOps(f(a)) 

    case class ParserOps[A](p: Parser[A]) {
        def |[B>:A](p2: Parser[B]): Parser[B] = self.or(p, p2)
        def or[B>:A](p2: Parser[B]): Parser[B] = self.or(p, p2)
    }                                  
}

In [ ]:
run(or(string("abra"),string("cadabra")))("abra") == Right("abra")

In [ ]:
run("abra" | "cadabra")("abra") == Right("abra")

-----

#### 하고 싶은 것
* 특정 문자열을 탐지하는 파서를 
* 한 문자열 입력에
* 여러번 적용하고 싶다.
    - "abraca"에 ('a')파서를 3번 적용 -> 'a', 'a', 'a'
    - "ababcad"에 ("ab" | "cad")파서를 3번 적용 -> "ab", "ab", "cad"

----

In [ ]:
// listOfN(3, 파서) -> 파서(리스트) 생성
def listOfN[A](n: Int, p: Parser[A]): Parser[List[A]]

In [ ]:
run(listOfN(3, "ab" | "cad")("ababcad") == Right(List("ab", "ab", "cad"))

In [ ]:
run(listOfN(3, "ab" | "cad")("cadabab") == Right(List("cad", "ab", "ab"))

In [ ]:
run(listOfN(3, "ab" | "cad")("ababab") == Right(List("ab", "ab", "ab"))

-----

#### 고려해 볼 만한 추가적인 파싱 과제들과 영감을 얻을 만한 질문 몇 가지
* 0개 이상의 문자 'a'들을 인식하고 그 개수를 돌려주는 Parser[Int].
    - 예를 들어 "aa"에 대해서는 2를 돌려줘야 하고
    - "aa"나 "b123"('a'로 시작하지는 문자열)에 대해서는 0을 돌려줘야 한다.
* 하나 이상의 문자 'a'들을 인식하고 그 개수를 돌려주는 Parser[Int].
    - 0개 이상의 'a'들을 인식하는 파서에 사용하는 것과 동일한 조합기들을 이 파서를 이용해서 구현할 수 있을까?
    - 'a'로 시작하지 않는 문자열은 인식하지 말아야 한다.
        - 그런 실패의 경우 오류를 어떻게 보고하는 것이 좋을까?
        - 실패의 경우에 "Expected one or more 'a'" 같은 명시적인 메시지를 제시하는 기능을 API에 추가해야 할까?
* ....(책에는 이밖에도 많다...도전~) 

# 9.2 가능한 대수 하나

* 9.2.1 슬라이싱과 비지 않은 되풀이

#### 하고 싶은 것
* 0개 이상의 문자 'a'를 인식해서 그 개수를 돌려주는 파서

----

In [ ]:
// many('a'파서) -> 파서(리스트) 생성
def many[A](p: Parser[A]): Parser[List[A]]

In [ ]:
def map[A,B](a: Parser[A])(f: A => B): Parser[B]

In [ ]:
// 이제 파서를 정의할 수 있게 되었다.
map(many(char('a')))(_.size)

In [ ]:
// ParserOps의 메서드로 추가하자. 그러면 아래처럼 깔끔한 구문을 쓸수 있게 된다.
val numA: Parser[Int] = char('a').many.map(_.size)

In [ ]:
run(numA)("aaa") == Right(3)

In [ ]:
run(numA)("b") == Right(0)

-----

#### 하고 싶은 것
* map은 Parser가 파싱에 성공하면 그냥 그 결과 값만 변환해야 한다.
    - map이 입력 문자들을 더 조사해서는 안되며,
    - 실패한 파서를 map이 성공으로 간주해서도 안된다(그 역도 마찬가지).
* 일반화하자면, Par나 Gen에 대한 map처럼 이 map도 구조를 보존해야 한다. 
* map(p)(a => a) == p

----

#### 목록 9.2 Parser와 map의 조합

In [ ]:
import fpinscala.testing._

trait Parsers[ParseError, Parser[+_]]
    ...
    object Laws {
        def equal[A](p1: Parser[A], p2: Parser[A])(in: Gen[String]): Prop =
            forAll(in)(s => run(p1)(s) == run(p2)(s))
        
        def mapLaw[A](p: Parser[A])(in: Gen[String]): Prop =
            equal(p, p.mam(a => a))(in)
    }

#### 이제 이렇게 구현해볼 수 있다.

In [ ]:
def char(c: Char): Parser[Char] =
    string(c.toString) map (_.charAt(0))

In [ ]:
def succeed[A](a: A): Parser[A] =
    string("") map (_ => a)

In [ ]:
run(succeed(a))(s) == Right(a)

## 9.2.1 슬라이싱과 비지 않은 되풀이

#### 하고 싶은 것
* map과 many을 조합해서 타겟 문자 개수를 세는 구현은, 비효율적이다.
    - List[Char]를 구축하고는 그 길이만 추출하고 값들을 폐기하므로 비효율적
* Parser가 입력 문자열에서 자신이 조사하는 부분만 보게 만들자.
* 이것이 비지 않은 되풀이(non-empty repetition)

----

In [ ]:
def slice[A](p: Parser[A]): Parser[String]

In [ ]:
// 원래 이 구현을
val numA: Parser[Int] = char('a').many.map(_.size)

// 이렇게 구현할 수 있다.
val numA: Parser[Int] = char('a').many.slice.map(_.size)

---

#### 하고 싶은 것
* 하나 이상의 'a'문자들을 인식해야 한다면 어떻게 할까?
* 비지 않은 되풀이 방식으로 구현해보자

----

In [ ]:
def many1[A](p: Parser[A]): Parser[List[A]]

In [ ]:
// many1을 기본수단으로 쓰는게 아니고,
// many를 이용해서 정의해야 한다.
// 그러면 한 파서를 실행하고 그것이 성공하면 또 다른 파서를 실행하는 수단이 필요하다.
def product[A,B](p: Parser[A], p2: Parser[B]): Parser[(A,B)]

#### ParserOps의 메서드로 추가해놓자. 중위표현을 위해.

In [ ]:
a product b

In [ ]:
// 이런 표현도 동치가 되도록 추가해놓자
a ** b 

----

In [ ]:
def many[A](p: Parser[A]): Parser[List[A]] =
    map2(p, many(p))(_ :: _) or succeed(List())

In [ ]:
// 어떤 파서 p에 대한 many(p)가 평가되는 과정을 추적한 결과의 일부분
map(p)
map2(p, many(p))(_ :: _)
map2(p, map2(p, many(p))(_ :: _))(_ :: _)
map2(p, map2(p, map2(p, many(p)))(_ :: _))(_ :: _)(_ :: _)
...

// map2는 항상 둘째 인수를 평가하므로, many 호출은 절대로 종료되지 않는다!
// product와 map2의 둘째 인수를 엄격하지 않게 만들어야 한다!

---

#### product와 map2의 둘째 인수를 엄격하지 않게 만들자

In [ ]:
def product[A,B](p: Parser[A], p2: => Parser[B]): Parser[(A,B)]

In [ ]:
def map2[A,B,C](p: Parser[A], p2: => Parser[B])(
                f: (A,B) => C): Parser[C] =
    product(p, p2) map (f.tupled)

----

#### or 조합기도 비엄격성 방식으로 다시 만들자.

In [ ]:
// 원래 버전
def or[A](p1: Parser[A], p2: Parser[A]): Parser[A]

In [ ]:
// 비엄격 버전
def or[A](p1: Parser[A], p2: => Parser[A]): Parser[A]

# 9.3 문맥 민감성 처리

#### 하고 싶은 것
* '4' 같은 어떤 숫자 하나 다음에 그 개수만큼의 문자 'a'가 오는 문자열을 파싱한다고 하자.
    - 예) "0", "1a", "2aa", "4aaaa" 
* 이런것을 문맥 민감 문법(context-sensitive grammer; 또는 문맥 감지 문법)이라고 한다.
* 지금까지 만든 것으로는 이를 처리할 수 없다.
* 표현력의 한계를 깨기 위해, 새 수단을 도입하자.

----

In [ ]:
def flatMap[A,B](p: Parser[A])(f: A => Parser[B]): Parser[B]

# 9.4 JSON 파서 작성

* 9.4.1 JSON 서식
* 9.4.2 JSON 파서

In [ ]:
def jsonParser[Err,Parser[+_]](P: Parsers[Err,Parser]): Parser[JSON] = {
    import P._
    val spaces = char(' ').many.slice
    ...
}

## 9.4.1 JSON 서식

In [ ]:
trait JSON
object JSON {
    case object JNull extends JSON
    case class JNumber(get: Double) extends JSON
    case class JString(get: String) extends JSON
    case class JBool(get: Boolean) extends JSON
    case class JArray(get: IndexedSeq[JSON]) extends JSON
    case class JObject(get: Map[String, JSON]) extends JSON
}

## 9.4.2 JSON 파서

# 9.5 오류 보고

* 9.5.1 가능한 설계 하나
* 9.5.2 오류의 중첩
* 9.5.3 분기와 역추적의 제어

## 9.5.1 가능한 설계 하나

In [ ]:
def label[A](msg: String)(p: Parser[A]): Parser[A]

In [ ]:
case class Location(input: String, offset: Int = 0) {
    lazy val line = input.slice(0, offset+1).count(_ == '\n') + 1
    lazy val col = input.slice(0, offset+1).lastIndexOf('\n') match {
        case -1 => offset + 1
        case lineStart => offset - lineStart
    }
}


def errorLocation(e: ParseError): Location
def errorMessage(e: ParseError): String

## 9.5.2 오류의 중첩

In [ ]:
val p = label("first magic word")("abra") ** 
        "".many ** 
        label("second magic word")("cadabra")

In [ ]:
def scope[A](msg: String)(p: Parser[A]): Parser[A]

In [ ]:
case class ParseError(stack: List[(Location,String)])

In [ ]:
trait Parsers[Parser[+_]] {
    def run[A](p: Parser[A])(input: String): Eiter[ParseError,A]
}

## 9.5.3 분기와 역추적의 제어

In [ ]:
val spaces = " ".many
val p1 = scope("magic spell") {
    "abra" ** spaces ** "cadabra"
}
val p2 = scope("gibberish") {
    "abba" ** spaces ** "babba"
}
val p = p1 or p2

In [ ]:
def attempt[A](p: Parser[A]): Parser[A]

In [ ]:
attemp(p flatMap (_ => fail)) or p2 == p2

In [ ]:
(attempt("abra" ** spaces ** "abra")) ** "cadabra") or (
    "abra" ** spaces "cadabra!")

# 9.6 대수의 구현

* 9.6.1 가능한 구현 하나
* 9.6.2 파서들의 순차 실행
* 9.6.3 파서에 이름표 붙이기
* 9.6.4 실패의 극복과 역추적
* 9.6.5 문맥 민감성 파싱

* strings(s) - 하나의 string을 인식해서 돌려준다.
* regex(s) - 정규표현식 s를 인식한다.
* slice(s) - 파싱 성공시 입력 중 p가 조사한 부분을 돌려준다.
* label(e)(p) - 실패 시 배정된 메시지를 e로 치환한다.
* scope(e)(p) - 실패 시 e를 p가 돌려준 스택에 추가한다.
* flatMap(p)(f) - 파서를 실행한 후 그 결과에 기초해서 둘째 파서를 선택, 실행한다.
* attempt(p) - p의 확정을 p의 성공 이후로 미룬다.
* or(p1,p2) - 두 파서 중 하나를 선택한다. 먼저 p1을 시도하고, 만일 p1이 입력에 대해서 미확정 상태에서 실패하면 p2를 시도한다.

## 9.6.1 가능한 구현 하나

In [ ]:
def string(s: String): Parser[A]

In [ ]:
def run[A](p: Parser[A])(input: String): Either[ParseError,A]

In [ ]:
type Parser[+A] = String => Either[ParseError,A]

In [ ]:
def string(s: String): Parser[A] =
  (input: String) =>
    if (input.startsWith(s))
        Right(s)
    else
        Left(Location(input).toError("Expected: " + s))

In [ ]:
def toError(msg: String): ParseError =
    ParseError(List((this, msg)))

## 9.6.2 파서들의 순차 실행

In [ ]:
type Parser[+A] = Location => Result[A]

trait Result[+A]
case class Sucess[+A](get: A, charsConsumed: Int) extends Result[A]
case class Failur(get: ParseError) extends Result[Nothing]

## 9.6.3 파서에 이름표 붙이기

In [ ]:
def push(loc: Location, msg: String): ParseError =
    copy(stack = (loc, msg) :: stack)

In [ ]:
def scope[A](msg: String)(p: Parser[A]): Parser[A] =
    s => p(s).mapError(_.push(s.loc,msg))

In [ ]:
def mapError(f: ParseError => ParseError) : Result[A] = this match {
    case Failure(e) => Failure(f(e))
    case _ => this
}

In [ ]:
scope(msg1)(a ** scope(msg2)(b))

In [ ]:
def label[A](msg: String)(p: Parser[A]): Parser[A] =
    s => p(s).mapError(_.label(msg))

In [ ]:
def label[A](s: String): ParseError =
    ParseError(latestLoc.map((_,s)).toList)

def latestLoc: Option[Location] =
    latest map (_._1)

def latest: Option[(Location,String)] =
    stack.lastOption

## 9.6.4 실패의 극복과 역추적

In [ ]:
case class Failure(get: ParseError,
                  isCommitted: Boolean) extends Result[Nothing]

In [ ]:
def attempt[A](p: Parser[A]): Parser[A] =
    s => p(s).uncommit

In [ ]:
def uncommit: Result[A] = this match {
    case Failure(e,true) => Failure(e,false)
    case _ => this
}

In [ ]:
def or[A](x: Parser[A], y: => Parser[A]): Parser[A] =
    s => x(s) match {
        case Failure(e, false) => y(s)
        case r => r
    }

## 9.6.5 문맥 민감성 파싱

#### 목록 9.3 파서를 확정 상태로 만드는 addCommit 메서드를 이용한 구현

In [ ]:
def flatMap[A,B](f: Parser[A])(g: A => Parser[B]): Parser[B] =
    s => f(s) match {
        case Success(a, n) => g(a)(s.advanceBy(n))
                                .addCommit(n != 0)
                                .advanceSuccess(n)
        case e@Failure(_,_) => e
    }

In [ ]:
def advanceBy(n: Int): Loation =
    copy(offset = offset + n)

In [ ]:
def addCommit(isCommitted: Boolean): Result[A] = this match {
    case Failure(e,c) => Failure(e, c || isCommitted)
    case _=> this
}

In [ ]:
def advanceSuccess(n: Int): Result[A] = this match {
    case Success(a,m) => Success(a, n+m)
    case _ => this
}

# 9.7 요약

# 참고자료

In [1]:
def map2[A,B,C](fa: List[A], fb: List[B])(f:(A,B)=>C): List[C] =
    fa flatMap(a => fb map(b => f(a,b)))

In [2]:
map2(List(1,2), List(3,4))(_+_)

List(4, 5, 5, 6)

In [3]:
List.map2(List(1, 2), List(3, 4)) (_ + _)

: 